In [1]:
# run this first to enable plotly offline plotting, then plot in a separate cell, else might get blank plots after saving
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

from IPython.display import HTML # scripts to hide input code and to autorun all cells at startup
HTML('''<script>code_show=true;function code_toggle(){if(code_show){$('div.input').hide();}else{$('div.input').show();} 
code_show=!code_show}$(document).ready(code_toggle);</script><a onClick="code_toggle()">Show / hide input code</a>
<script>require(["base/js/namespace","jquery"],function(a,b){b(a.events).on("kernel_ready.Kernel",function()
{a.actions.call("jupyter-notebook:run-all-cells-below")})});</script>''')

In [2]:
import pandas as pd
import plotly.graph_objs as go
from ipywidgets import *
from IPython.display import display, clear_output
from time import time

# load data from file
cols_acc_info = ['Account','Type','Currency']
col_bal = 'BalanceEOD_CAD'
data = pd.read_csv('data_merged.csv', usecols = ['Date']+cols_acc_info+[col_bal])
data['Date'] = pd.to_datetime(data['Date'].astype(str))
data = data.drop_duplicates()
data = data.sort_values('Date') # required for merge_asof later

start = data['Date'].min() # min date of all accounts
end = data['Date'].max() # max date of all accounts
delta = (end-start).days

acc_info = data[cols_acc_info].drop_duplicates().set_index('Account') # account information
acc_info = acc_info.assign(k=acc_info.index.str.lower()).sort_values('k').drop('k',1) # sort by account names in lowercase
acc_info['All'] = acc_info.index + acc_info['Type'] + acc_info['Currency'] # combine all info (for searching purpose)
g_date = data.groupby('Account')['Date']
acc_info['Date_min'] = g_date.first() # first date for each account
acc_info['Date_max'] = g_date.last() # last date for each account
acc_names = acc_info.index.tolist() # list of all account names

current_filter = '' # filter string, to be updated from the search box
acc_names_f = acc_names # filtered account names, to be updated on_submit of search box
data_f = data # filtered accounts' data, to be updated on_submit of search box

# lookup table for frequency options
freq = pd.DataFrame({'code':['1D','2D','3D','4D','5D','W','SMS','MS','QS','Q','AS','A'],
                     'unit':['days']*8 + ['months']*4,
                     'upto':[1, 1, 2, 3, 4, 6, 14, 30, 2, 0, 11, 0]}, 
                     index= ['1 day','2 days','3 days','4 days','5 days','week', 'semi-month',
                             'month','quarter','quarter-end','year','year-end'])
# style for plot
colors = ['#1F77B4','#FF7F0E','#2CA02C','#D62728','#9575D2','#8C564B','#E377C0','#7F7F7F','#F9D00F','#17BECF',
          '#295BA7','#FC993C','#1B8057','#B90B0B','#8C54A1','#779977','#F0AE2C','#55A44E','#35BCBF','#FF9898']
acc_info['Color'] = ((len(acc_info)/len(colors) + 1)*colors)[0:len(acc_info)] # assign colors to accounts (repeat if needed)
plot_layout = go.Layout(width=980,height=450,margin=go.Margin(l=50,r=215,b=50,t=0,pad=4),hovermode='closest',showlegend=True)
line_total = dict(shape='linear', width=1.5, color='#1F77B4') # line style for the total line
def line_style(color_code): return dict(shape='linear', width=1, color=color_code)

def int2date(x): return start + pd.DateOffset(x)
def int2datestr(x): return int2date(x).strftime('%Y-%m-%d')
def date2str(d): return d.strftime('%Y-%m-%d')

def width(x): return dict(width=str(x)+'px')
def widthleft(x,y): return dict(width=str(x)+'px',left=str(y)+'px')
def space(x): return Label(layout=width(x))

def update_plot():
    t0 = time()
    
    # clear previous plot (wait for new output before clearing)
    clear_output(wait=True)

    # generate reporting dates
    shift = pd.DateOffset(months=wOffset.value) if freq['unit'][wFreq.value]=='months' else pd.DateOffset(days=wOffset.value)
    dates = pd.date_range(int2date(wRange.value[0]),int2date(wRange.value[1]),freq=freq['code'][wFreq.value]) + shift
    dates = pd.DataFrame(dates,columns=['Date'])
    if len(dates)==0: return
    
    # prepare plot data on reporting dates
    accs = pd.DataFrame(acc_names_f, columns=['Account'])
    base = pd.merge(dates.assign(k=0), accs.assign(k=0), on='k').drop('k',1) # broadcast dates to all accounts
    base = base.join(acc_info[['Date_min','Date_max']], on='Account') # add columns 'Date_min' and 'Date_max'
    base = base[(base['Date']>=base['Date_min']) & (base['Date']<=base['Date_max'])] # remove dates out of min-max range
    base = base.drop(['Date_min','Date_max'], axis=1)
    plot_data = pd.merge_asof(base,data_f,on='Date',by='Account').fillna(0) # merge nearest balance to base dates
    plot_accs = sorted(base['Account'].drop_duplicates().tolist(), key=lambda s: s.lower()) # name of accounts to plot
    if len(plot_accs)==0: return
    
    # calculate total and make plot
    g = plot_data.groupby('Account')
    traces = [go.Scatter(x = g.get_group(a)['Date'].tolist(), y = g.get_group(a)[col_bal].tolist(), name = a,
                         fill='tozeroy',mode='lines',line=line_style(acc_info['Color'][a])) for a in plot_accs]
    if wTotal.value == True:
        y_total = plot_data.groupby('Date')[col_bal].sum().tolist()
        x_total = plot_data['Date'].drop_duplicates().tolist()
        traces += [go.Scatter(x=x_total,y=y_total,name='Total balance',mode='lines',line=line_total)]
    fig = go.Figure(data=traces,layout=plot_layout)
    iplot(fig)
    #print 'Load time: {:.2f}s'.format(time()-t0)

def wRange_move(x):
    wStart.value = int2datestr(wRange.value[0])
    wEnd.value = int2datestr(wRange.value[1])
    update_plot()
def wStart_submit(x):
    try:
        newstart = max(pd.to_datetime(wStart.value),start)
        wStart.value = date2str(newstart)
        wRange.value = [(newstart-start).days,wRange.value[1]]
    except:
        wStart.value = int2datestr(wRange.value[0])
def wEnd_submit(x):
    try:
        newend = min(pd.to_datetime(wEnd.value),end)
        wEnd.value = date2str(newend)
        wRange.value = [wRange.value[0],(newend-start).days]
    except:
        wEnd.value = int2datestr(wRange.value[1])
def wFreq_move(x):
    wOffset.value = 0
    wOffset.max = freq['upto'][wFreq.value]
    wOffset.min = -freq['upto'][wFreq.value]
    l5.value = 'shift ('+ freq['unit'][wFreq.value] +')'
    update_plot()    
def wOffset_move(x):
    update_plot()
def wFilter_submit(x):
    global current_filter, acc_names_f, data_f
    current_filter = wFilter.value.strip()
    words = current_filter.split()
    acc_names_f = [a for a in acc_names if all(w in acc_info['All'][a] for w in words)] if len(words)>0 else acc_names
    data_f = data[data['Account'].isin(acc_names_f)]
    update_plot()
def wClear_click(x):
    wFilter.value = ''
    if current_filter != '':
        wFilter_submit(x)
def wTotal_click(x):
    update_plot()

# create widgets
wRange = IntRangeSlider(value=[0,delta],min=0,max=delta,readout=False,continuous_update=False,layout=width(380))
wStart = Text(value = date2str(start),layout=width(87))
wEnd = Text(value = date2str(end),layout=width(87))
wFreq = widgets.SelectionSlider(value='week',options=freq.index.tolist(),continuous_update=False,layout=width(300))
wOffset = IntSlider(min=-1,max=1,continuous_update=False,layout=width(240))
wFilter = Text(placeholder='Search by name, type etc.',layout=width(180))
wClear = Button(description='x',layout=width(25),button_style='success')
wTotal = Checkbox(value=True,layout=width(30))

# specify widgets' responses
wRange.observe(wRange_move,'value')
wStart.on_submit(wStart_submit)
wEnd.on_submit(wEnd_submit)
wFreq.observe(wFreq_move,'value')
wOffset.observe(wOffset_move,'value')
wFilter.on_submit(wFilter_submit)
wClear.on_click(wClear_click)
wTotal.observe(wTotal_click,'value')

# text labels
l1 = Label('Date range', layout=width(70))
l2 = Label('from', layout=widthleft(40,10))
l3 = Label('to', layout=widthleft(26,8))
l4 = Label('Frequency', layout=width(70))
l5 = Label('shift (days)', layout=widthleft(115,30))
l6 = Label('Search', layout=width(50))
l7 = Label('show total balance', layout=width(150))

# display widgets in box containers
box1 = Box(children=[l1,wRange,l2,wStart,l3,wEnd,space(70),wFilter,wClear], layout=Layout(display='flex', width='100%'))
box2 = Box(children=[l4,wFreq,l5,wOffset,space(36),wTotal,l7], layout=box1.layout)
display(box1, box2)

update_plot()

IOError: File dataJ_merged.csv does not exist